In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
import tensorflow_datasets as tfds

In [70]:
class ZeroMask():
    """
    when this class is initialized, it will will initilize a mask. When the instance is called an input 
    array the same shape as the mask will be multiplied with the mask and returned. The returned array will
    be masked to 0 at indecies specified in the "zero_indices" input. 
    
    Having this as a call able class, lets us repeativly use a mask multiple times in a cleanly abstracted way.
    """
    def __init__(self, shape, zero_indices):
        """
        inputs:
            shape: shape of matrix, ei (channels, filters, kernals, x_dims, y_dims)
            zero_indices: list of tuples that specify indexes to be masked to 0, ei [(row, col, channel, filter)]
                if zero_indices is equal to [(r, col, ch, f)] then mask[r, col, ch, f] will be set to 0
        instance variables
        """
        self.mask = np.ones(shape)
        for index in zero_indices:
            self.mask[index] = 0.  
    def __call__(self, arr):
        assert arr.shape == self.mask.shape, "Incorrect dims, mask shape: {}, arr shape{}".format(self.mask.shape, arr.shape)
        return self.mask * arr


In [71]:
mask_func = ZeroMask(shape = (3, 3, 1, 7), zero_indices = [(0, 0, 0, 0), (2, 2, 0, 0), (2, 0, 0, 0), (0, 2, 0, 0)])

In [72]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [112]:
#custom conv layer
class IregConv2D(tf.keras.Model): # still needs to figure out backprop, keep kernal wieghts at 0!!.
    def __init__(self, zero_mask, filters, kernel_size, *args, **kwargs):
        super(IregConv2D, self).__init__()
        self.conv = Conv2D(filters = filters, kernel_size = kernel_size, *args, **kwargs)
        self.zero_mask = zero_mask
    def call(self, x):
        return self.conv(x)
    def build(self, input_shape):
        super(IregConv2D, self).build(input_shape)
        weights = self.conv.get_weights()
        weights[0] = self.zero_mask(weights[0]) # set weights to 0 here
        self.conv.set_weights(weights)
        print(self.get_weights())
    def get_conv(self):
        return self.conv
    
## simple mnist network

class TestNet(tf.keras.Model):
    def __init__(self):
        super(TestNet, self).__init__()
        self.conv = IregConv2D(mask_func, filters = 7, kernel_size = (3, 3), activation = 'relu')
        self.flatten = Flatten()
        self.dense = Dense(32, activation = 'relu')
        self.head = Dense(10, activation = 'softmax')
    
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.head(x)
        return x

In [113]:
x = tf.keras.Input(shape = (28, 28))
model = TestNet()

In [114]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [115]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y, training):
    # training=training is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    y_ = model(x, training=training)

    return loss_object(y_true=y, y_pred=y_)
optimizer = Adam()

In [116]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 1

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    # Training loop - using batches of 32
    for x, y in ds_train:
        # Optimize the model
        loss_value, grads = grad(model, x, y)
        grads[0] = mask_func(grads[0]) ###this is the important part here!!
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Track progress
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy.update_state(y, model(x, training=True))

    # End epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

[array([[[[-0.        ,  0.2583604 , -0.06744356,  0.07393333,
           0.08499187,  0.0345456 ,  0.26686954]],

        [[ 0.16667655,  0.21842569,  0.2051959 ,  0.11701426,
          -0.09301087, -0.21842128, -0.25044236]],

        [[ 0.        , -0.1593098 , -0.13538757,  0.12862182,
           0.03625143,  0.1379807 ,  0.00132874]]],


       [[[ 0.19055638,  0.25506288,  0.04876301, -0.07887872,
          -0.2645274 , -0.22109315, -0.07971083]],

        [[ 0.25541568, -0.0160954 ,  0.12995818,  0.0807198 ,
          -0.2753123 ,  0.0581347 , -0.23939303]],

        [[ 0.04594359, -0.00056967,  0.2708329 , -0.17134649,
          -0.0814804 , -0.2593218 , -0.08769596]]],


       [[[-0.        , -0.18986523, -0.17994967, -0.15135096,
          -0.01983565,  0.256046  , -0.08410347]],

        [[ 0.27390897, -0.10320137, -0.00517026,  0.1962828 ,
           0.25061077,  0.2574649 ,  0.06720707]],

        [[-0.        ,  0.14445561,  0.28609192, -0.17702708,
          -0.20515537

tf.Tensor(
[[[[-0.00000000e+00 -1.70008421e-01 -7.54460171e-02 -1.36452496e-01
    -1.54245785e-02 -3.43228132e-02 -1.65037084e-02]]

  [[-1.25313014e-01 -1.66545570e-01 -1.11122742e-01 -1.32046923e-01
    -1.05286976e-02 -5.79053126e-02 -1.73539156e-04]]

  [[-0.00000000e+00 -1.25135183e-01 -1.35227323e-01 -1.19358376e-01
    -1.05692465e-02 -6.91911057e-02 -3.99219524e-03]]]


 [[[-1.56355590e-01 -1.64138198e-01 -1.11951455e-01 -9.47211683e-02
    -1.26663782e-03 -7.83466473e-02 -1.03022056e-02]]

  [[-1.33367538e-01 -1.45137817e-01 -1.49436101e-01 -8.14428031e-02
    -5.92178258e-05 -9.68498290e-02 -8.16008262e-03]]

  [[-1.14478707e-01 -9.81660932e-02 -1.59672230e-01 -6.72826841e-02
    -4.24910977e-05 -1.00234576e-01 -2.65163053e-02]]]


 [[[-0.00000000e+00 -1.10625826e-01 -1.32738531e-01 -4.37287726e-02
    -1.66391060e-02 -1.26253843e-01 -2.10138038e-02]]

  [[-1.15311317e-01 -9.04672593e-02 -1.72720373e-01 -2.82132197e-02
    -1.21799856e-02 -1.42520592e-01 -2.86663137e-02]]

 

tf.Tensor(
[[[[-0.13916841 -0.20609142 -0.0538162  -0.17392716 -0.03215959
    -0.07134042 -0.01705373]]

  [[-0.14607055 -0.20155859 -0.105952   -0.19515494 -0.01632323
    -0.08510041 -0.00131404]]

  [[-0.12184878 -0.16129051 -0.14341928 -0.17737529 -0.01629901
    -0.07462834 -0.01922714]]]


 [[[-0.19669028 -0.21639428 -0.11354886 -0.14790775 -0.0018324
    -0.11940573  0.00025485]]

  [[-0.19190052 -0.20044461 -0.1758458  -0.14971322 -0.0005002
    -0.11876032  0.00293817]]

  [[-0.1642609  -0.16826445 -0.19781137 -0.12969455 -0.00085833
    -0.10507122 -0.0386401 ]]]


 [[[-0.20503119 -0.17139131 -0.1490668  -0.08848646 -0.02314051
    -0.1867575  -0.01172412]]

  [[-0.19557121 -0.15151867 -0.20580165 -0.08182108 -0.02228701
    -0.18775326 -0.03470959]]

  [[-0.1755716  -0.12928025 -0.21186703 -0.06379674 -0.01479755
    -0.17015812 -0.08802865]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[-0.         -0.20609142 -0.0538162  -0.17392716 -0.03215959
    -0.07134042 -0.0

tf.Tensor(
[[[[-1.25463337e-01 -1.54887348e-01 -5.01303934e-02 -1.36526152e-01
    -2.97065265e-02 -4.50658277e-02 -1.95523202e-02]]

  [[-1.44687504e-01 -1.51026100e-01 -9.74916518e-02 -1.65271908e-01
    -1.75105892e-02 -7.84930736e-02 -7.87799619e-03]]

  [[-1.27155721e-01 -1.20302536e-01 -1.26631051e-01 -1.58199146e-01
    -1.91576779e-02 -7.08437786e-02 -3.35171260e-02]]]


 [[[-1.55075163e-01 -1.43343836e-01 -8.76661837e-02 -1.16073661e-01
    -3.37189459e-03 -8.92220512e-02  5.85006783e-04]]

  [[-1.64048493e-01 -1.32976905e-01 -1.36039644e-01 -1.35262549e-01
    -1.13321657e-04 -1.21798277e-01 -1.16510047e-02]]

  [[-1.41875029e-01 -1.16194993e-01 -1.52127877e-01 -1.15341038e-01
    -1.17920805e-04 -1.15887791e-01 -6.60127923e-02]]]


 [[[-1.45010352e-01 -1.13470659e-01 -9.31377858e-02 -8.84754956e-02
    -4.59614694e-02 -1.43122807e-01 -1.11790933e-02]]

  [[-1.54169589e-01 -1.06866822e-01 -1.46502391e-01 -8.98059011e-02
    -3.76252085e-02 -1.67035460e-01 -4.98322733e-02]]

 

tf.Tensor(
[[[[-0.0655995  -0.11384997 -0.0576046  -0.09122033 -0.03091218
    -0.02387799 -0.00108994]]

  [[-0.07632278 -0.09871401 -0.10137017 -0.10696322 -0.02179318
    -0.04105441  0.01175094]]

  [[-0.07458499 -0.06126305 -0.1137453  -0.09236564 -0.02521107
    -0.04304739 -0.00864576]]]


 [[[-0.10163519 -0.1264615  -0.08346894 -0.09122951 -0.00686216
    -0.06205247  0.00651475]]

  [[-0.10646538 -0.10508853 -0.12950924 -0.08707985 -0.00068058
    -0.07322839  0.00653028]]

  [[-0.09666525 -0.07880178 -0.14634755 -0.05999214 -0.00086207
    -0.06633221 -0.02986849]]]


 [[[-0.12724072 -0.11651057 -0.09657517 -0.06866208 -0.03408604
    -0.09930964 -0.00758976]]

  [[-0.13058248 -0.1049986  -0.14953688 -0.0599063  -0.0266596
    -0.11216609 -0.02336041]]

  [[-0.11789755 -0.08960116 -0.17044964 -0.0359696  -0.01393517
    -0.09076392 -0.06129783]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[-0.         -0.11384997 -0.0576046  -0.09122033 -0.03091218
    -0.02387799 -0.

...

tf.Tensor(
[[[[ 0.00391461 -0.00912807 -0.01531679 -0.01615699  0.0952158
    -0.00282406 -0.03137226]]

  [[-0.00562709 -0.01310983 -0.02120319 -0.02333036  0.083648
    -0.00465663 -0.01403717]]

  [[-0.00835838 -0.00528326 -0.02400199 -0.01641619  0.0995587
     0.00484702 -0.01983852]]]


 [[[ 0.00130163 -0.01212907 -0.01359496 -0.01481449  0.05570573
    -0.00755103 -0.01259118]]

  [[-0.01099053 -0.01371708 -0.02212298 -0.01787756  0.05850553
    -0.01285095 -0.01097051]]

  [[-0.01039977 -0.00796292 -0.02726611 -0.00880277  0.08765967
    -0.00884931 -0.02744291]]]


 [[[ 0.00130899 -0.01648267 -0.01807593 -0.01575517  0.0413265
    -0.00970124 -0.01721365]]

  [[-0.00879152 -0.01889062 -0.02782276 -0.01415249  0.06041216
    -0.01889319 -0.02749387]]

  [[-0.01066745 -0.01529879 -0.03461818 -0.00860537  0.08059818
    -0.01991737 -0.04423557]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[ 0.         -0.00912807 -0.01531679 -0.01615699  0.0952158
    -0.00282406 -0.03137

tf.Tensor(
[[[[-0.01001352 -0.00790784 -0.01072633  0.00350019 -0.00106996
    -0.01364714  0.00525126]]

  [[-0.01440828 -0.00780252 -0.02162902 -0.00515692 -0.03258323
    -0.01605185  0.00187499]]

  [[-0.01742613 -0.00134333 -0.02377726 -0.00770755 -0.02704133
    -0.02299662  0.0040829 ]]]


 [[[-0.007353   -0.00265225 -0.0061261   0.00201199  0.02556031
    -0.01591158  0.01054946]]

  [[-0.00913335 -0.00203383 -0.01728719 -0.00541843 -0.00274529
    -0.02200633  0.00396422]]

  [[-0.01225227  0.00267869 -0.01830122 -0.00459321 -0.00922611
    -0.02189764 -0.00293098]]]


 [[[-0.0071059   0.00243228  0.00013183  0.00146209 -0.02547724
    -0.01663449  0.02943156]]

  [[-0.0036738   0.0044687  -0.00731981 -0.00424813 -0.06007126
    -0.02380951  0.02509179]]

  [[-0.00473764  0.00836221 -0.00866741 -0.00237833 -0.0891505
    -0.01993737  0.0141314 ]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[-0.         -0.00790784 -0.01072633  0.00350019 -0.00106996
    -0.01364714  0.

tf.Tensor(
[[[[ 0.0254822   0.00268275  0.01158985  0.0051775  -0.01769745
     0.02326445  0.015557  ]]

  [[ 0.02186172 -0.00339533  0.00730225 -0.006874   -0.00282331
     0.02406605  0.01365274]]

  [[ 0.012272   -0.0001324   0.00455669 -0.01006774  0.01996155
     0.02870572  0.02331079]]]


 [[[ 0.00729269 -0.0120652   0.00064993 -0.00655103 -0.00165694
     0.02098086  0.02294521]]

  [[-0.00441781 -0.0148668  -0.0047388  -0.01346054  0.03491881
     0.01952036  0.02129531]]

  [[-0.00969117 -0.00031651 -0.0046408  -0.01485039  0.06532799
     0.01249281  0.00919625]]]


 [[[-0.00711108 -0.01443734 -0.00839945 -0.00899996  0.00863092
     0.0173945   0.01688294]]

  [[-0.01659309 -0.0108467  -0.00895495 -0.01532692  0.06559546
     0.01517274  0.01384905]]

  [[-0.01627475  0.00240989 -0.0120991  -0.01402806  0.10613273
     0.00856727  0.00122841]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[ 0.          0.00268275  0.01158985  0.0051775  -0.01769745
     0.02326445  0

tf.Tensor(
[[[[-0.02511755 -0.01860208 -0.0235028   0.02812295  0.16007149
     0.01106211 -0.02952186]]

  [[-0.03949982 -0.02413171 -0.03810616  0.01646731  0.12021022
    -0.0072433  -0.00770207]]

  [[-0.04601236 -0.01996912 -0.05002514  0.00714856  0.14217573
    -0.02472847 -0.04339962]]]


 [[[-0.01521549 -0.0036999  -0.02155411  0.02705328  0.12010957
     0.01623749 -0.01456654]]

  [[-0.03208803 -0.01403215 -0.03394219  0.01458514  0.09290589
    -0.00996    -0.01639973]]

  [[-0.0415959  -0.01821404 -0.04912509  0.00676213  0.12568818
    -0.02265118 -0.0768591 ]]]


 [[[-0.00211304  0.01009263 -0.01084751  0.02378734  0.1339966
     0.02511233 -0.03744314]]

  [[-0.01593583 -0.00178774 -0.01868857  0.01297444  0.13845764
    -0.00295132 -0.06896844]]

  [[-0.02522913 -0.00153495 -0.03010942  0.00376617  0.13587952
    -0.01779789 -0.11311142]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[-0.         -0.01860208 -0.0235028   0.02812295  0.16007149
     0.01106211 -0.

tf.Tensor(
[[[[-0.01089256  0.0047249   0.00284178  0.02395399 -0.03679017
     0.00948394  0.02708428]]

  [[-0.01758306 -0.00280725 -0.00138626  0.03655989 -0.03684434
     0.0023242   0.01938102]]

  [[-0.02233167 -0.00610543  0.00146374  0.03882252 -0.01616044
    -0.00434577  0.01204461]]]


 [[[ 0.00117146  0.01198914  0.01877805  0.02186871 -0.00577759
     0.00921215  0.02267309]]

  [[-0.00103121  0.01130502  0.01176283  0.03353123  0.0023427
     0.0061607   0.02225424]]

  [[-0.00250089  0.00028662  0.01007761  0.04065751  0.00193496
     0.00507501  0.01950604]]]


 [[[ 0.01413459  0.01385211  0.02389465  0.01647531 -0.04655089
     0.01215433  0.02378407]]

  [[ 0.01534049  0.00999869  0.01772012  0.02122744 -0.04825669
     0.00838621  0.02376891]]

  [[ 0.01269411 -0.00562333  0.01077774  0.02674115 -0.04430749
     0.00968217  0.02835767]]]], shape=(3, 3, 1, 7), dtype=float32)
tf.Tensor(
[[[[-0.          0.0047249   0.00284178  0.02395399 -0.03679017
     0.00948394  0.

In [95]:
def x(*args):
    print(*args)
x(1, 1)

1 1


In [109]:
x = (2, 3)
print(*x)

2 3
